In [ ]:
!pip install datasets transformers accelerate tokenizers

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel,DataCollatorForLanguageModeling,Trainer,TrainingArguments

from datasets import load_dataset

In [ ]:
dataset=load_dataset("text",data_files="/content/dr_patient.txt")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 78
    })
})

In [ ]:
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
def tokenize_function(examples):
  tokenizer_inputs=tokenizer(examples["text"],truncation=True,padding="max_length",max_length=128)
  tokenizer_inputs["labels"]=tokenizer_inputs["input_ids"].copy()
  return tokenizer_inputs

In [ ]:
tokenized_dataset=dataset.map(tokenize_function,batched=True,num_proc=4,remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/78 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 78
    })
})


In [ ]:
model=GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)

In [ ]:
training_args=TrainingArguments(
    output_dir="./dr_patient_finetuned",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2
)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"]
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.8706470489501953, metrics={'train_runtime': 29.4094, 'train_samples_per_second': 13.261, 'train_steps_per_second': 3.4, 'total_flos': 25475973120000.0, 'train_loss': 0.8706470489501953, 'epoch': 5.0})